# Step 1) Creating the dictionary(vocabulary) from the articles

In [2]:
#importing the os library to get the required folders 
import os
from os import listdir

In [3]:
#my_path is the variable to contain the destination
my_path = r'20_newsgroups'
folders = os.listdir(my_path) #this gives the list of all the folders present in the 20_newsgroups
folders

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
#here sub_folders_path will contain the final locations of all the articles
categories_path=[] #this will contain the location to all the folders
j=0
sub_folders_path=[]  #this will contain the final locations of all the folders
for current_folder in folders:
    categories_path.append(my_path + '\\' + current_folder) #appending the location
    current_folder_sub_folders = os.listdir(categories_path[j]) #this gives the list of all the sub folders
    subpaths = []
    for k in current_folder_sub_folders:
        subpaths.append(my_path + '\\' + current_folder + '\\'+ k)
    sub_folders_path.append(subpaths)
    j += 1




In [5]:
#for finding stop_words
import nltk  #this libaray contains all the inbuilt stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english') #it gives a dictionary of all the stopwords
stop_words={}
for x in stopwords:
    if x in stop_words:
        stop_words[x]+=1
    else:
        stop_words[x]=1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Naman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#buildidng vocabulary
vocabulary={}
for i in range(0 , len(sub_folders_path)):
    for j in range(0 , len(sub_folders_path[i])):
        with open(sub_folders_path[i][j] , 'r') as f: #opening the file 
            data=f.read()
            data=data.split() #creating the list of all the words present in the particualr file 
            for x in data:
                if x not in stop_words:
                    if x in vocabulary:
                        vocabulary[x]+=1
                    else:
                        vocabulary[x]=1


In [7]:
#sorting the vocabulary into a list
import operator
sorted_vocab = sorted(vocabulary.items(), key=operator.itemgetter(1)) #I have sorted according to the increasing  occurrences of of words
len(sorted_vocab)

463145

In [8]:
#this is the count of words to keep
k=len(sorted_vocab)-2000
k

461145

In [9]:
# creating final vocab with a range of 2000 words
final_vocab={}
while k<len(sorted_vocab):
    final_vocab[sorted_vocab[k][0]]=sorted_vocab[k][1]
    k+=1
# now we have got our final vocab of 2000 words

In [10]:
len(final_vocab)  #printing the len of final_vcab

2000

# Step 2) a) creating features from the dictionary and putting it into a 2-d array for training and testing

In [11]:
#building the features in data for further training and testing
X=[] # this will contain our final data on which we have to train and test
for i in range(0 , len(sub_folders_path)):
    for j in range(0 , len(sub_folders_path[i])):
        with open(sub_folders_path[i][j] , 'r') as f:
            
            data=f.read()
            data=data.split()   #all the words of the file have been created as a list of words 
            lst=[0 for i in range(2000)]  #since we are taking 2000 features
            data_dict={}
            for x in data:    #creating a dictionary of words since dictionaries are faster to access
                if x in data_dict:
                    data_dict[x]+=1
                else:
                    data_dict[x]=1
                    
            index=0          #writing down the count of words
            for x in final_vocab:
                if x in data_dict:
                    lst[index]+=data_dict[x]
                index+=1
                
            X.append(lst)   #appending the data point
#so our final data in the form of a 2-d array is ready on which we have to work .            

# Step 2)b) building Y according to the classes

In [12]:
# now we have to Build Y which will contain the target classes corresonding to x
Y=[]
index=0
for i in range(0 , len(sub_folders_path)):
    l=len(sub_folders_path[i])
    for j in range(0 , l):
        Y.append(folders[index])   #appending the corresponding classes 
    index+=1

In [13]:
len(Y) #this is the length of our Y which is the target

19997

In [14]:
#importing the sklearn libarary
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test=train_test_split(X , Y , random_state=1)  #splitting the data (random_state is to ensure we get similar kind of data everytime)

# Step 3) Now train and test data on inbuilt Multinomial NaiveBayes

In [15]:
#import the required things from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score  #thises parameters will help us compare later

In [16]:
clf=MultinomialNB() #creating an algo of MultinomialNB

In [17]:
clf.fit(X_train , Y_train) #fitting the training data on algo

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
Y_pred=clf.predict(X_test) #predicting the values on X_test

In [19]:
inbuilt_training_score=clf.score(X_train ,Y_train) # checking the accuracy on training data

In [20]:
inbuilt_testing_score=clf.score(X_test , Y_test)   #checking the accuaracy on testing data

# Step 4) Implementing self built Multinomial Naive Bayes

# Step 4)a) creating the fit function for X_train and Y_train

In [21]:
def fit_data(X_train , Y_train):
    different_classes=set(Y_train)  #this will give the different classes
    result={} 
    for current_class in different_classes:
        result["total data"]=len(Y_train)  #this gives the total number of rows
        result[current_class]={}    #this dictionary will contain the 
        current_class_rows=(Y_train==current_class)   #this will generate the required number of rows of a particular class
        X_train_new=X_train[current_class_rows]
        Y_train_new=Y_train[current_class_rows]
        result[current_class]["total count"]=len(Y_train_new)
        num_features=X_train_new.shape[1]
        for i in range(0 , num_features):
            result[current_class][i]=X_train_new[: , i].sum()   #keeping a count of all the words in a particular class
    return result
        

# Step 4)b) creating the probabilty function to check the probability of a data point belonging to a particular class

In [22]:
def probability(dictionary , current_class , x):
    
    total_classes=dictionary["total data"]   #this is the number of rows in the dataset. It contains all classes
    
    output=np.log(dictionary[current_class]["total count"])-np.log(total_classes)  #using log so as to avoid getting too small numbers
    
    num_features=len(dictionary[current_class].keys())-1   #this will give the number of features
    
    total_words_in_current_class=0
    for y in dictionary[current_class]:
        if y=="total count":
            continue
            
        total_words_in_current_class+=dictionary[current_class][y]   #this give the sum of all words
        
    total_words_in_current_class+=num_features
            
        
    for i in range(0 , num_features):
        if x[i]>0:       #this condition is to check whther the word exists in the vocabulary or not
            count_of_word=dictionary[current_class][i]+1
            word_probability=np.log(count_of_word)-np.log(total_words_in_current_class)
            output+=word_probability
        
    return output        # this is the final probabilty of an article belonging to a particular class
         
    
    
    

# Step4)c) getting the best class

In [23]:
def predict_class(dictionary , x):
    classes=dictionary.keys()   #this give us all the distinct classes 
    best_prob=-1000
    best_class=""
    first_run=True     #this is to ensure the assignment of a value to best_prob_and best_class during the first run
    for current_class in classes:
        if current_class=="total data":
            continue
        p_current_class=probability(dictionary , current_class , x)  #calling the probabilty function to calculate the probability of each class
        if first_run or p_current_class>best_prob:
            best_prob=p_current_class
            best_class=current_class
        first_run=False
    return best_class   #return the best class with the highest probability for an article belonging to a particular class
        

# Step4)d) predicting the final values for testing data

In [24]:
def predict_output(dictionary , X_new_test):
    Y_pred=[]  #this will contain our final predictions for testing data
    for x in X_new_test:
        ans=predict_class(dictionary , x)  #calling the function predict_class to get the required class
        Y_pred.append(ans)
    return Y_pred  #

In [25]:
#importing the numpy library foir mathematical calculations
import numpy as np

In [26]:
#converting the previous arrays into numpy arrays for easier calculations
X_new=np.array(X)
Y_new=np.array(Y)

In [27]:
#splitting the data (random state is used to get the similar type of data each time we run the program)
X_new_train , X_new_test , Y_new_train , Y_new_test=train_test_split(X_new , Y_new , random_state=1) 

In [28]:
#calling the fit function to return the dictionary containing all the info
dictionary=fit_data(X_new_train , Y_new_train)

In [29]:
#calling the predict_output function to get final output
Y_new_pred=predict_output(dictionary , X_new_test)
self_test_score=accuracy_score(Y_new_pred , Y_new_test) #calculating the accuarcy of our predictions

# Step5) Comparison between the inbuilt multinomial naive bayes and self implemented naive bayes

In [31]:
print("The accuracy of inbulit multinomial naive bayes on testing data is" , inbuilt_testing_score)
print("The accuracy of self implemented multinomial naive bayes on testing data is " , self_test_score)

The accuracy of inbulit multinomial naive bayes on testing data is 0.7888
The accuracy of self implemented multinomial naive bayes on testing data is  0.8094
